# Query Rewrite and Web Search Context for LLMs

- This code snippet demonstrates how to rewrite a query, perform a web search, and prepare the results for use with a language model (LLM). It includes functions to rewrite the query using a language model, perform a web search using the rewritten query, and format the search results for the LLM.





In [1]:
import sys
import os
from openai import AzureOpenAI
import requests
import json
from urllib.parse import urljoin

from azure.ai.projects import AIProjectClient
from azure.identity import DefaultAzureCredential
from langchain_core.prompts import PromptTemplate
from langchain_core.prompts import load_prompt
import sys
import logging
from utils.search_utils import web_search
from IPython.display import Markdown, display
from datetime import datetime
import time
import os
import pytz
from dotenv import load_dotenv
load_dotenv(override=True) 

True

In [2]:
# Configure logging
logging.basicConfig(level=logging.ERROR, format='%(asctime)s - %(name)s - %(levelname)s - %(message)s')
logger = logging.getLogger(__name__)


# Get credentials from environment variables
BING_GROUNDING_PROJECT_ENDPOINT = os.getenv("BING_GROUNDING_PROJECT_ENDPOINT")
BING_GROUNDING_CONNECTION_ID = os.getenv("BING_GROUNDING_CONNECTION_ID")
BING_GROUNDING_AGENT_MODEL_DEPLOYMENT_NAME = os.getenv("BING_GROUNDING_AGENT_MODEL_DEPLOYMENT_NAME")
BING_GROUNDING_MAX_RESULTS = int(os.getenv("BING_GROUNDING_MAX_RESULTS", 10))
BING_GROUNDING_MARKET = os.getenv("BING_GROUNDING_MARKET", "ko-KR")
BING_GROUNDING_SET_LANG = os.getenv("BING_GROUNDING_SET_LANG", "ko-KR")
GOOGLE_API_KEY = os.getenv("GOOGLE_API_KEY")
GOOGLE_CSE_ID = os.getenv("GOOGLE_CSE_ID")
AZURE_OPENAI_CHAT_DEPLOYMENT_NAME = os.getenv("AZURE_OPENAI_CHAT_DEPLOYMENT_NAME", "gpt-4o-mini")

# Web search mode: "google" or "bing"
web_search_mode = "bing"


client = AzureOpenAI(
  azure_endpoint = os.getenv("AZURE_OPENAI_ENDPOINT"), 
  api_key=os.getenv("AZURE_OPENAI_API_KEY"),  
  api_version="2024-08-01-preview"
)

current_dir = os.getcwd()

def rewrite_query_for_search(query, client: AzureOpenAI):
    
        
        prompt_path = os.path.join(current_dir, "prompts", "rewrite_prompt.yaml")
        QUERY_REWRITE_PROMPT = load_prompt(prompt_path, encoding="utf-8")
        response = client.chat.completions.create(
            model=AZURE_OPENAI_CHAT_DEPLOYMENT_NAME,
            messages=[
                {"role": "system", "content": QUERY_REWRITE_PROMPT.format(
                  user_query=query)},
                {"role": "user", "content": query}
            ],
            temperature=0.8,
            max_tokens=300,
            response_format= {"type": "json_object"},
        )
        
        return json.loads(response.choices[0].message.content.strip())


In [3]:

# 웹 검색 결과를 활용해 LLM 답변을 생성하는 비동기 함수
async def process_web_search(RESULTS_COUNT, input, web_search_mode=None, product_name=None):
    
    start_time = time.time()
    print(f"Original Input: {input}")

    # 검색 모드가 지정되지 않으면 환경변수
    if web_search_mode is None:
        web_search_mode = os.getenv("WEB_SEARCH_MODE", "google").lower()

    print(f"############## Web Search Mode: {web_search_mode}")
    # query rewrite (검색용/LLM용)
    query_rewrite = rewrite_query_for_search(input, client)
    print(f"Web Search Query: {query_rewrite['search_query']}")
    print(f"LLM Query: {query_rewrite['llm_query']}")

    # 웹 검색 실행 (비동기)
    print("Search rewrited search queries...")   
    results = await web_search(query_rewrite, RESULTS_COUNT, web_search_mode=web_search_mode, product_name=product_name)

    current_date = datetime.now(tz=pytz.timezone("Asia/Seoul")).strftime("%Y-%m-%d")


    # Get the current working directory for this notebook
    current_dir = os.getcwd()
    prompt_path = os.path.join(current_dir, "prompts", "generate_prompt.yaml")
    GENERATE_PROMPT = load_prompt(prompt_path, encoding="utf-8")
    
    answer_messages = [
        {"role": "system", "content": GENERATE_PROMPT.format(
            product_name=product_name,
            date=current_date,
            contexts=results if results else '검색 결과 없음',
            user_query=query_rewrite['llm_query'],
        )},
        {"role": "user", "content": query_rewrite['llm_query']}
    ]
            
    # 검색 결과가 리스트(정상)인 경우 LLM 호출, 아니면 안내 메시지 출력
    
    print("Generate response...")   
    
    if web_search_mode == "google" and isinstance(results, list) and results:
        response = client.chat.completions.create(
            model=AZURE_OPENAI_CHAT_DEPLOYMENT_NAME,
            messages=answer_messages,
            top_p=0.9,
            max_tokens=1500
        )
        display(Markdown(response.choices[0].message.content))
    elif web_search_mode == "bing" and isinstance(results, list) and results:
        for result in results:
            if isinstance(result, str):
                display(Markdown(result))
            else:
                display(Markdown(str(result)))

    else:
        display(Markdown("검색 결과가 없습니다. 다른 질문을 해주세요."))
    
    end_time = time.time()
    print(f"elapsed time: {end_time - start_time:.2f} seconds\n\n")


In [4]:
RESULTS_COUNT = 5

inputs = [
    "삼성전자 제품 중 2구 말고 다른 인덕션 추천해줘",
    # "부모님에게 선물하고 싶은데 삼성전자 TV 추천해줘",
    # "삼성전자 25년 제품이 작년 대비 좋아진것은",
    # "삼성전자 JBL과 하만카돈 차이점이 뭐야",
    # "갤럭시 버즈 이어버드 한쪽을 새로 구매했는데 페어링 어떻게 하나요",
    # "삼성전자 S25 무게가 S24와 비교 했을때 얼마나 차이나"
]

web_search_mode = "bing"

for input in inputs:
    await process_web_search(RESULTS_COUNT, input, web_search_mode, product_name="삼성전자")  # product_name은 필요에 따라 변경 가능

Original Input: 삼성전자 제품 중 2구 말고 다른 인덕션 추천해줘
############## Web Search Mode: bing


Web Search Query: 삼성전자 3구 이상 인덕션 추천
LLM Query: 삼성전자 인덕션 제품 중에서 2구 모델이 아닌 다른 인덕션 제품을 추천해 주세요. 특히 3구 이상 또는 다양한 화구 수를 가진 모델에 대해 주요 기능과 장점을 설명해 주시면 좋겠습니다.
Search rewrited search queries...
Generate response...


2025년 삼성전자 3구 이상 인덕션 모델 추천 및 주요 기능과 장점에 대해 다음과 같이 정리하였습니다.

# 삼성전자 3구 이상 인덕션 모델 추천 및 주요 기능

## 1. 삼성전자 3구 인덕션 모델 특징
- 3구 이상 인덕션은 한 번에 여러 요리를 효율적으로 할 수 있어 바쁜 가정이나 요리 공간이 넓은 곳에 적합합니다.
- 삼성 인덕션은 스마트 센서, 자동 화력 조절, 사용자 맞춤형 요리 설정 등의 첨단 기능이 탑재되어 있습니다.
- 삼성전자 인덕션은 세련된 디자인과 함께 사용자의 안전을 위한 과열 방지 및 자동 꺼짐 기능도 강화되어 있습니다.

## 2. 주요 모델 추천 (예시)
- **삼성 BESPOKE 3구 인덕션**: 맞춤형 디자인과 다양한 크기로 부엌 인테리어와 잘 어울리며, 스마트 IoT 연동으로 핸드폰 앱을 통해 제어 가능.
- **삼성스마트 인덕션 4구 모델**: 다수의 화구(쿡탑)로 동시에 다양한 음식을 조리할 수 있으며, 인공지능 화력 제어로 에너지 효율이 우수함.

## 3. 주요 기능 및 장점
- **스마트 화력 자동 조절**: 요리 종류나 냄비의 크기에 따라 화력을 자동으로 조절하여 음식이 타는 것을 방지하며 효율적 조리가 가능.
- **과열 방지 및 안전성 강화**: 과열 감지 센서가 작동하여 안전 사고를 예방하며, 어린이 보호 잠금 기능으로 안전성을 높임.
- **와이드 및 멀티존 쿡탑**: 와이드 화구 혹은 멀티존 기능을 통해 다양한 크기와 모양의 조리기구를 자유롭게 사용할 수 있음.
- **에너지 효율 최적화**: 인덕션 중심의 열 전달로 에너지 손실을 줄여 전기요금 절감에 효과적임.
- **스마트 홈 연동 및 원격 제어**: 스마트폰 앱과 연동해 인덕션 상태 확인 및 원격 조절, 예약 요리 설정이 가능하여 편리함 극대화.

## 4. 활용 팁
- 3구 또는 4구 이상 모델은 공간과 요리 스타일에 맞춰 선택하며, 가족 수가 많거나 다중 요리를 자주 할 경우 3구 이상을 추천.
- IoT 기능이 탑재된 모델은 스마트 홈 환경 구축 시 각종 기기와 동기화해 효율적인 관리 가능.

---

### 참고
- 삼성전자 공식 웹사이트에서 최신 모델과 상세 기능, 가격 정보를 확인할 수 있습니다.  
- 현재 2025년형 최신 3구 및 다구 모델들은 BESPOKE 라인업 및 스마트 IoT 연동 모델군이 가장 인기입니다.

더 구체적 모델 정보 및 구매는 삼성전자 공식 사이트 방문을 권장드립니다:  
[삼성전자 공식 인덕션 제품 페이지](https://www.samsung.com/sec/kitchen/cooking-appliances/induction-cooktops/)

---

이와 같이 삼성전자 3구 이상 인덕션 모델은 다양한 화구 수와 스마트 기능으로 요리를 더 편리하고 효율적으로 만들어 주며, 안전성과 에너지 절약 면에서도 뛰어난 장점을 가지고 있습니다.🍳✨

# 【답변 1:0†source】

elapsed time: 22.81 seconds


